<a href="https://colab.research.google.com/github/MikeIV2007/DS_HW_07/blob/main/DS_HW_07_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнє завдання частина 1



На цьому тижні ми вивчили як працюють рекомендаційні системи. Пропонуємо вам познайомитись з бібліотекою surprise, котра якраз є по суті додатком до знайомої нам бібліотеки scikit-learn для тренування моделей рекомендаційних систем.



Візьміть датасет movielens і побудуйте модель матричної факторизації. У даній бібліотеці він має назву SVD. Підберіть найкращі параметри за допомогою крос-валідації, також поекспериментуйте з іншими алгоритмами розрахунків (SVD++, NMF) і оберіть той, який буде оптимальним.



Підказки як саме побудувати дану модель ви знайдете в документації до даної бібліотеки.







# Load Data Set

In [ ]:
pip install scikit-surprise

In [ ]:
# https://surprise.readthedocs.io/en/stable/dataset.html
import surprise

In [ ]:
from surprise import Dataset
data = Dataset.load_builtin('ml-100k')

In [ ]:
import pandas as pd
df = pd.DataFrame(data.raw_ratings, columns=['user_id', 'item_id', 'rating', 'timestamp'])
df.drop(columns=['timestamp'], inplace=True)
display(df)

,user_id,item_id,rating
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0
...,...,...,...
99995,880,476,3.0
99996,716,204,5.0
99997,276,1090,1.0
99998,13,225,2.0


In [ ]:
from surprise import Dataset, Reader

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'item_id', 'rating']], reader)

# 5 folds Automatic cross-validation:

In [ ]:
from surprise import SVD, SVDpp, NMF
from surprise.model_selection import cross_validate

svd_model = SVD()
svd_results = cross_validate(svd_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

svdpp_model = SVDpp()
svdpp_results = cross_validate(svdpp_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

nmf_model = NMF()
nmf_results = cross_validate(nmf_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

print("SVD Results:")
for key, values in svd_results.items():
    if key != 'fit_time' and key != 'test_time':
        print(f"{key}: {sum(values) / len(values)}")

print("\nSVD++ Results:")
for key, values in svdpp_results.items():
    if key != 'fit_time' and key != 'test_time':
        print(f"{key}: {sum(values) / len(values)}")

print("\nNMF Results:")
for key, values in nmf_results.items():
    if key != 'fit_time' and key != 'test_time':
        print(f"{key}: {sum(values) / len(values)}")


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9386  0.9319  0.9336  0.9310  0.9459  0.9362  0.0055  
MAE (testset)     0.7384  0.7368  0.7365  0.7325  0.7425  0.7373  0.0032  
Fit time          1.86    1.74    2.12    2.26    1.85    1.97    0.19    
Test time         0.14    0.14    0.36    0.14    0.14    0.19    0.09    
Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9214  0.9230  0.9239  0.9155  0.9194  0.9207  0.0030  
MAE (testset)     0.7230  0.7253  0.7244  0.7181  0.7204  0.7223  0.0026  
Fit time          26.89   28.62   27.51   26.75   26.99   27.36   0.68    
Test time         6.03    5.39    5.71    6.03    5.45    5.72    0.27    
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (tests

**Висновок:** Найкраще спрацював SVD++ метод

# Train-test split and the fit() method

In [ ]:
from surprise import accuracy, Dataset, SVD, SVDpp, NMF
from surprise import SVD, SVDpp, NMF
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25)

algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)
svd_accuracy = accuracy.rmse(predictions)
print (f"svd_accuracy: {svd_accuracy}\n")

algo = SVDpp()
algo.fit(trainset)
predictions = algo.test(testset)
svdpp_accuracy = accuracy.rmse(predictions)
print (f"svdpp_accuracy: {svdpp_accuracy}\n")

algo = NMF()
algo.fit(trainset)
predictions = algo.test(testset)
mnf_accuracy = accuracy.rmse(predictions)
print (f"mnf_accuracy: {mnf_accuracy}\n")

RMSE: 0.9352
svd_accuracy: 0.9352400523341436

RMSE: 0.9191
svdpp_accuracy: 0.9190802142839041

RMSE: 0.9602
mnf_accuracy: 0.9601744314461195



Висновок: В обох випадках найкраще спрацював SVD++ метод